In [10]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../raw')
# sys.path.clear()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import json, csv
import numpy as np
import pandas as pd

TRAIN_PATH = '../raw/train.csv'

In [59]:
# train_csv = list(csv.reader(open(TRAIN_PATH)))
# train_csv

train_df = pd.read_csv(TRAIN_PATH)
train_df[20:40:2]

,id,keyword,location,text,target
20,31,NaN,NaN,this is ridiculous....,0
22,33,NaN,NaN,Love skiing,0
24,36,NaN,NaN,LOOOOOOL,0
26,38,NaN,NaN,Was in NYC last week!,0
28,40,NaN,NaN,Cooool :),0
30,44,NaN,NaN,The end!,0
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
36,54,ablaze,Pretoria,@PhDSquares #mufc they've built so much hype a...,0
38,56,ablaze,NaN,Barbados #Bridgetown JAMAICA ÛÒ Two cars set ...,1


## handle missing values in a naive way

In [29]:
fill_v = {'keyword': 'missing', 'location': 'unknown'}
filled_df = train_df.fillna(value=fill_v)
filled_df.head()

,id,keyword,location,text,target
0,1,missing,unknown,Our Deeds are the Reason of this #earthquake M...,1
1,4,missing,unknown,Forest fire near La Ronge Sask. Canada,1
2,5,missing,unknown,All residents asked to 'shelter in place' are ...,1
3,6,missing,unknown,"13,000 people receive #wildfires evacuation or...",1
4,7,missing,unknown,Just got sent this photo from Ruby #Alaska as ...,1


In [37]:
X = filled_df.drop(['target'], axis=1)
y = filled_df['target']
X.shape

(7613, 4)

## simple linear SVC - baseline

In [82]:
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer

class LabeledNormalizer(Normalizer):
    def fit(self, X, *args, **kwargs):
        try:
            self.names = X.columns
        except:
            self.names = [str(i) for i in range(X.shape[1])]
        return super().fit(X, *args, **kwargs)
        
    def get_feature_names(self):
        return self.names
    
vec = ColumnTransformer([
    ('norm', LabeledNormalizer(), ['id']),
    ('kw', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b",), 'keyword'),
    ('loc', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b"), 'location'),
    ('text', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b"), 'text'),
])

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
clf = SVC(kernel='linear', probability=True, verbose=True)
# clf = LinearSVC(verbose=True)


pipeline = make_pipeline(vec, clf)

## evaluation

In [65]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict

def evaluate(_clf, X, y):
    report = classification_report(
        y_true=y, y_pred=cross_val_predict(pipeline, X, y, cv=5)
    )
    print(report)
    scores = cross_val_score(_clf, X, y, scoring='accuracy', cv=10)
    print('Cross-validation Accuracy: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
    
    _clf.fit(X,y)
    print('Training Set Accuracy: {:.3f}'.format(_clf.score(X,y)))


evaluate(pipeline, X, y)

              precision    recall  f1-score   support

           0       0.67      0.69      0.68      4342
           1       0.57      0.54      0.55      3271

    accuracy                           0.63      7613
   macro avg       0.62      0.62      0.62      7613
weighted avg       0.62      0.63      0.63      7613

Cross-validation Accuracy: 0.515 ± 0.238
Training Set Accuracy: 0.942


In [68]:
from sklearn.externals import joblib
import os

pipeline.fit(X,y)
baseline_mdl = '../models/baseline-linear-svc.pkl'
joblib.dump(pipeline, baseline_mdl)
!ls -lSh $baseline_mdl

[LibSVM]-rw-r--r-- 1 hongyu hongyu 7.3M Feb 24 14:58 ./models/baseline-linear-svc.pkl
